## Overview

[Website](https://www.cellpose.org) | [GitHub](https://github.com/mouseland/cellpose) | [Paper](https://www.biorxiv.org/content/10.1101/2025.04.28.651001v1)

In this section, we’ll learn how to use **Cellpose**, a powerful deep learning tool for cell segmentation. Cellpose works on a wide variety of microscopy images—including both nuclei and full cells—and doesn’t require retraining for many common use cases.

We’ll start by running Cellpose on a single image and then move to batch processing and visualization. Later, we’ll explore how to customize parameters and interpret Cellpose outputs like masks and flow fields.

> 💡 Tip: If you're not using a GPU (or are on a Mac with Apple Silicon), we recommend running this notebook on **Google Colab** for faster performance.

## Make sure you have GPU access

To Enable GPU:

1. navigate to `Runtime -> Change Runtime Type`
2. select `Python 3` as `Runtime Type`
3. select one available GPU (e.g. `T4 GPU`) as `Hardware accelerator`.

<br>

<div align="left"> <img src="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/images/cellpose/colab_runtime.png" alt="Ilastik Logo" width="400"></div>


## Mount your google drive

To access the data for the course you first need to mount your Google Drive.

Run the cell below to connect your Google Drive to colab and follow the instructions to authenticate your Google account.

You will need to allow access to your Google Drive so that the notebook can read and write files.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")


Then click on `folder icon` on the left bar, press the `refresh button`. Your Google Drive folder should now be available here (e.g. MyDrive).

<div align="left"> <img src="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/images/cellpose/colab_folder.png" alt="Ilastik Logo" width="300"></div>

## Download the Data

Run the cell below to download the data for this exercise and save it in you Google Drive. A new folder called `bobiac_data_cellpose` will be created in your Google Drive.

In [ ]:
# Create directory
!mkdir -p /content/bobiac_data_cellpose
# Download the data
!wget https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/data/05_segmentation_cellpose.zip -O /content/bobiac_data_cellpose/05_segmentation_cellpose.zip
# Unzip the data, remove zip file and macOS metadata files (if any)
!cd /content/bobiac_data_cellpose && unzip 05_segmentation_cellpose.zip && rm -f 05_segmentation_cellpose.zip && rm -rf __MACOSX

## Install Cellpose


In [ ]:
# !pip install cellpose


## Import Libraries

## Setup

In [ ]:
io.logger_setup()  # run this to get printing of progress

use_gpu = core.use_gpu()
if use_gpu:
    print("Using GPU for Cellpose")
else:
    raise ImportError("No GPU access, change your runtime as explained above.")

## Run Cellpose on A Sample Image

In this section, we’ll apply Cellpose to a single image and visualize the segmentation result.

You’ll see how easy it is to segment cells using just a few lines of code. We’ll use the `model.eval()` function to make predictions, and then display:
- The original image
- The predicted segmentation mask
- The outlines of detected cells

We’ll also look at the `flows` output, which helps Cellpose group pixels into individual objects.

No model training required—just load, run, and view the results.

### Load the Image

All the data for this exercise is stored in the `bobiac_data_cellpose` folder on your Google Drive created in the previous steps. You can get the path to the files inside this folder by clicking on the `folder icon` on the left bar, then right-clicking on the file and selecting `Copy path`.

### Initialize the Model

To initialize Cellpose model we can use the [`models.CellposeModel()`](https://cellpose.readthedocs.io/en/latest/api.html#cellposemodel) class.

Among the parameters we can set if we want to use `gpu` (if available) as well which **model** to use. The current default is Cellpose-SAM as `cpsam` (another example is `cyto3`, the previous U-Net-based model).

If it is the first time you run this notebook, the model will be downloaded automatically. This may take a while.

### Run Cellpose

After initializing the model, we can run it on the image using the [`model.eval()`](https://cellpose.readthedocs.io/en/latest/api.html#id0) method.

Here are some of the parameters you can change:

* ***flow_threshold*** is  the  maximum  allowed  error  of  the  flows  for  each  mask.   The  default  is 0.4.
    *  **Increase** this threshold if cellpose is not returning as many masks as you’d expect (or turn off completely with 0.0)
    *   **Decrease** this threshold if cellpose is returning too many ill-shaped masks.

* ***cellprob_threshold*** determines proability that a detected object is a cell.   The  default  is 0.0.
    *   **Decrease** this threshold if cellpose is not returning as many masks as you’d expect or if masks are too small
    *   **Increase** this threshold if cellpose is returning too many masks esp from dull/dim areas.

* ***tile_norm_blocksize*** determines the size of blocks used for normalizing the image. The default is 0, which means the entire image is normalized together.
  You may want to change this to 100-200 pixels if you have very inhomogeneous brightness across your image.


### Display the Results

## Run Cellpose on a Folder of Images

Now that we’ve seen how to run Cellpose on a single image, let’s scale up and apply it to a **folder of images**. This is useful when you have an entire experiment or dataset that you want to segment automatically.

Cellpose provides a convenient built-in function for this: `model.eval()` can be used with a folder path, and it will process all compatible image files in that folder.

We’ll walk through how to:
- Point Cellpose to your image directory
- Choose the segmentation type (e.g., "nuclei" or "cyto")
- Set output options like saving masks and outlines
- Visualize a few results to make sure everything worked

> 💡 Tip: Make sure your images are in `.tif`, `.png`, or `.jpg` format.

Let’s get started with folder-based segmentation!
